In [1]:
import numpy as np

import pandas as pd

import collections

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import accuracy_score

In [3]:
videos_df = pd.read_csv("/Users/neelimajagtap/Desktop/SJSU /Sem2/DATA245/Project/DAtaset/archive (7)/USvideos.csv", header=0)

videos_df.head()


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [4]:
category_json = pd.read_json("/Users/neelimajagtap/Desktop/SJSU /Sem2/DATA245/Project/DAtaset/archive (7)/US_category_id.JSON")

category_json.head()

,kind,etag,items
0,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
1,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
2,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
3,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
4,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."


In [5]:
category_dic = [{'id': item['id'], 'title': item['snippet']['title']} for item in category_json['items']]

category_dic

[{'id': '1', 'title': 'Film & Animation'},
 {'id': '2', 'title': 'Autos & Vehicles'},
 {'id': '10', 'title': 'Music'},
 {'id': '15', 'title': 'Pets & Animals'},
 {'id': '17', 'title': 'Sports'},
 {'id': '18', 'title': 'Short Movies'},
 {'id': '19', 'title': 'Travel & Events'},
 {'id': '20', 'title': 'Gaming'},
 {'id': '21', 'title': 'Videoblogging'},
 {'id': '22', 'title': 'People & Blogs'},
 {'id': '23', 'title': 'Comedy'},
 {'id': '24', 'title': 'Entertainment'},
 {'id': '25', 'title': 'News & Politics'},
 {'id': '26', 'title': 'Howto & Style'},
 {'id': '27', 'title': 'Education'},
 {'id': '28', 'title': 'Science & Technology'},
 {'id': '29', 'title': 'Nonprofits & Activism'},
 {'id': '30', 'title': 'Movies'},
 {'id': '31', 'title': 'Anime/Animation'},
 {'id': '32', 'title': 'Action/Adventure'},
 {'id': '33', 'title': 'Classics'},
 {'id': '34', 'title': 'Comedy'},
 {'id': '35', 'title': 'Documentary'},
 {'id': '36', 'title': 'Drama'},
 {'id': '37', 'title': 'Family'},
 {'id': '38', '

In [6]:
category_df = pd.DataFrame(category_dic)

category_df = category_df.rename(index=str, columns={"id": "Category_ID", "title": "Category"})

category_df.head()

,Category_ID,Category
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports


In [8]:
columns = ['title','category_id']

new_videos = videos_df[columns]

new_videos.to_csv("new_video.csv", index=False)

new_videos = pd.read_csv("new_video.csv", header=0, names=['Title','Category_ID'])

In [9]:
vector = CountVectorizer()

counts = vector.fit_transform(new_videos['Title'].values)

In [10]:
nb = MultinomialNB()

targets = new_videos['Category_ID'].values

nb.fit(counts,targets)

MultinomialNB()

In [11]:
X = counts

y = targets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .1)

mn = MultinomialNB().fit(X_train, y_train)

nb_predictions = mn.predict(X_test)

In [12]:
acc_nb = mn.score(X_test, y_test)

print('Accuracy', acc_nb)

Accuracy 0.8940170940170941


In [59]:
titles = [
        "Best winter fashion ",
        "Olympics opening ceremony highlights",
        "CNN world news on donald trump",
        "Ed Sheeran - Perfect (Official Music Video)",
        "how to do eyeshadow",
          "cute dogs",
          "India"
         ]


In [60]:
titles_counts = vector.transform(titles)

predict = nb.predict(titles_counts)

predict


array([22, 17, 25, 10, 26, 15, 19])

In [61]:
category_list = []

for category_id in predict:

    matching = [x for x in category_dic if x["id"] == str(category_id)]

    if matching:
    
      category_list.append(matching[0]["title"])

In [62]:
title_dfs = []

for i in range(0, len(titles)):
  
   title_category = {'Title': titles[i],  'Category': category_list[i]}

   title_dfs.append(title_category)

In [63]:
df = pd.DataFrame(predict)

title_df = pd.DataFrame(title_dfs)

pre_final_df = pd.concat([df, title_df], axis=1)

pre_final_df.columns = (['Categ_ID', 'Video Title', 'Predicted Video Type'])

result_df = pre_final_df.drop(['Categ_ID'],axis=1)

cols = result_df.columns.tolist()

cols = cols[-1:] + cols[:-1]

result_df = result_df[cols]

result_df


,Predicted Video Type,Video Title
0,People & Blogs,Best winter fashion
1,Sports,Olympics opening ceremony highlights
2,News & Politics,CNN world news on donald trump
3,Music,Ed Sheeran - Perfect (Official Music Video)
4,Howto & Style,how to do eyeshadow
5,Pets & Animals,cute dogs
6,Travel & Events,India
